<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open('css/custom.css', 'r').read()
    return HTML(styles)
css_styling()

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2017 &middot; Uhan</h5>

<h1 class='lesson_title'>Quiz 10 &mdash; 18 April 2017</h1>

## Instructions

This quiz is due **at the beginning of class on Tuesday 25 April**.

Type your answers to all of the problems **directly in this Jupyter notebook**, and submit this notebook (just the `ipynb` file) using the [SA367 submission form](https://script.google.com/a/macros/usna.edu/s/AKfycbyieji_Lk5-LCNNq1GKJyY9jC71-XoBZ_RHjCSrjqBKsSEhRZ0c/exec).

**You may work with other midshipmen, but each midshipman must submit his or her own quiz.** You may use any of your own course materials (e.g., notes, homework) as well as any materials linked from the course website.

## Your name

_Write your name here, please! Double-click to edit._

## Who did you work with?

_Write the name of your collaborators here. Double-click to edit._

## Problem 1

Place your code in the cell below.

In [ ]:
# Setup code
from stochasticdp import StochasticDP

# Total number of possessions
# Drive Averages: 2 * (#Dr) / (G * (# of teams))
T = 23

# 1-pt. conversion success probabilities
# Kicking and Punting: XP%
pA = 0.936
pB = 0.936

# 2-pt. conversion success probabilities
# Scoring Offense: 2PM / 2PA
qA = 0.486
qB = 0.486

# 1-pt. vs 2-pt attempts
# 1-pt.: Scoring Offense: XPA / (XPA + 2PA)
b1 = 0.919
b2 = 1 - b1

# Possession outcome probabilities - Team A
# TD: (Scoring Offense: ATD) / (Drive Averages: #Dr)
# FG: (Scoring Offense: FGM) / (Drive Averages: #Dr)
tA = 0.218
gA = 0.145
zA = 1 - tA - gA

# Possession outcome probabilities - Team B
tB = 0.218
gB = 0.145
zB = 1 - tB - gB

# Probability that Team A wins in OT
r = 0.5

# Number of stages
number_of_stages = T + 1

# Maximum lead for Team A
max_d = 20

# List of states
states = []
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-max_d, max_d + 1):
            states.append((n, k, d))
            
# List of decisions
decisions = [1, 2, 'none']

# Initialize stochastic dynamic program - we want to maximize, so minimize = False
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

# Transition probabilities from (A, 6, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        # 1-point conversion
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 6, d), t, 1] += (1 - pA) * tB
        dp.transition[('B', 6, max(d - 5, -max_d)), ('A', 6, d), t, 1] += pA * tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 6, d), t, 1] += (1 - pA) * gB
        dp.transition[('B', 3, max(d - 2, -max_d)), ('A', 6, d), t, 1] += pA * gB
        dp.transition[('B', 0, d),                  ('A', 6, d), t, 1] += (1 - pA) * zB
        dp.transition[('B', 0, min(d + 1, max_d)),  ('A', 6, d), t, 1] += pA * zB
        
        # 2-point conversion
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 6, d), t, 2] += (1 - qA) * tB
        dp.transition[('B', 6, max(d - 4, -max_d)), ('A', 6, d), t, 2] += qA * tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 6, d), t, 2] += (1 - qA) * gB
        dp.transition[('B', 3, max(d - 1, -max_d)), ('A', 6, d), t, 2] += qA * gB
        dp.transition[('B', 0, d),                  ('A', 6, d), t, 2] += (1 - qA) * zB
        dp.transition[('B', 0, min(d + 2, max_d)),  ('A', 6, d), t, 2] += qA * zB
        
# Transition probabilities from (A, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 3, d), t, 'none'] += tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 3, d), t, 'none'] += gB
        dp.transition[('B', 0, d),                  ('A', 3, d), t, 'none'] += zB
        
# Transition probabilities from (A, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('B', 6, max(d - 6, -max_d)), ('A', 0, d), t, 'none'] += tB
        dp.transition[('B', 3, max(d - 3, -max_d)), ('A', 0, d), t, 'none'] += gB
        dp.transition[('B', 0, d),                  ('A', 0, d), t, 'none'] += zB

# Transition probabilities from (B, 6, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('A', 6, min(d + 6, max_d)), ('B', 6, d), t, 'none'] += (1 - b1*pB - b2*qB) * tA
        dp.transition[('A', 3, min(d + 3, max_d)), ('B', 6, d), t, 'none'] += (1 - b1*pB - b2*qB) * gA
        dp.transition[('A', 0, d),                 ('B', 6, d), t, 'none'] += (1 - b1*pB - b2*qB) * zA
        
        dp.transition[('A', 6, min(d + 5, max_d)),  ('B', 6, d), t, 'none'] += b1 * pB * tA
        dp.transition[('A', 3, min(d + 2, max_d)),  ('B', 6, d), t, 'none'] += b1 * pB * gA
        dp.transition[('A', 0, max(d - 1, -max_d)), ('B', 6, d), t, 'none'] += b1 * pB * zA
        
        dp.transition[('A', 6, min(d + 4, max_d)),  ('B', 6, d), t, 'none'] += b2 * qB * tA
        dp.transition[('A', 3, min(d + 1, max_d)),  ('B', 6, d), t, 'none'] += b2 * qB * gA
        dp.transition[('A', 0, max(d - 2, -max_d)), ('B', 6, d), t, 'none'] += b2 * qB * zA

# Transition probabilities from (B, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('A', 6, min(d + 6, max_d)), ('B', 3, d), t, 'none'] += tA
        dp.transition[('A', 3, min(d + 3, max_d)), ('B', 3, d), t, 'none'] += gA
        dp.transition[('A', 0, d),                 ('B', 3, d), t, 'none'] += zA

# Transition probabilities from (B, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-max_d, max_d + 1):
        dp.transition[('A', 6, min(d + 6, max_d)), ('B', 0, d), t, 'none'] += tA
        dp.transition[('A', 3, min(d + 3, max_d)), ('B', 0, d), t, 'none'] += gA
        dp.transition[('A', 0, d),                 ('B', 0, d), t, 'none'] += zA

# Transition probabilities from (A, 6, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    # 1-point conversion
    dp.transition[('A', 6, min(d + 1, max_d)),  ('A', 6, d), T - 1, 1] += pA
    dp.transition[('A', 6, d),                  ('A', 6, d), T - 1, 1] += 1 - pA

    # 2-point conversion
    dp.transition[('B', 0, min(d + 2, max_d)),  ('A', 6, d), T - 1, 2] += qA
    dp.transition[('B', 0, d),                  ('A', 6, d), T - 1, 2] += (1 - qA)

# Transition probabilities from (A, 3, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('A', 3, d), ('A', 3, d), T - 1, 'none'] += 1

# Transition probabilities from (A, 0, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('A', 0, d), ('A', 0, d), T - 1, 'none'] += 1

# Transition probabilities from (B, 6, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('B', 6, max(d - 2, -max_d)),  ('B', 6, d), T - 1, 'none'] += qB * b2
    dp.transition[('B', 6, max(d - 1, -max_d)),  ('B', 6, d), T - 1, 'none'] += pB * b1
    dp.transition[('B', 6, d),                  ('B', 6, d), T - 1, 'none'] += 1 - pB * b1 - qB * b2

# Transition probabilities from (B, 3, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('B', 3, d), ('B', 3, d), T - 1, 'none'] += 1

# Transition probabilities from (B, 0, d) in stage T - 1
for d in range(-max_d, max_d + 1):
    dp.transition[('B', 0, d), ('B', 0, d), T - 1, 'none'] += 1
    
# Boundary conditions
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-max_d, max_d + 1):
            if d > 0:
                dp.boundary[n, k, d] = 1
            elif d == 0:
                dp.boundary[n, k, d] = r
            else:
                dp.boundary[n, k, d] = 0

# Solve the stochastic dynamic program
value, policy = dp.solve()

## Problem 2

Place your code in the cell below.

In [ ]:
value[0, ('A', 3, 3)]

## Problem 3

Place any code that helps you answer the problem in the cell below.

In [ ]:
d = -4
for t in range(number_of_stages - 1):
    print("Points ahead: {1}   Possession: {0}    Go for: {2}   Pr(win): {3}".format(t, d, policy[t, ('A', 6, d)], value[t, ('A', 6, d)]))

_Write your response here. Double-click to edit._

## Problem 4

_Write your response here. Double-click to edit._

## When you're finished

* Select **Cell &#8594; Run All**  to run all the cells from top to bottom. Make sure your notebook runs as you expect before submitting it.